<a href="https://colab.research.google.com/github/Altemir1/Process-Mining/blob/main/conformance_checking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
log = ""

with open("eventlog.txt", "r") as file:
  log = file.read()

log

In [ ]:
log = log.split('\n')

for i in range(len(log)):
  trace = log[i].split(" ")
  log[i] = trace

log

In [ ]:
!pip install pm4py

In [5]:
from pm4py.objects.log.obj import EventLog, Trace, Event

def convert_to_event_log(traces):
    log = EventLog()
    for trace in traces:
        t = Trace()
        for idx, activity in enumerate(trace):
            e = Event()
            e["concept:name"] = activity
            e["time:timestamp"] = idx
            t.append(e)
        log.append(t)
    return log

traces = log
event_log = convert_to_event_log(traces)

In [8]:
from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.algo.conformance.tokenreplay import algorithm as token_replay
import numpy as np


net, initial_marking, final_marking = alpha_miner.apply(event_log)
results = token_replay.apply(event_log, net, initial_marking, final_marking)

# Extract and compute metrics
trace_fitnesses = []
missing_tokens = []
consumed_tokens = []
remaining_tokens = []
produced_tokens = []

for r in results:
    trace_fitnesses.append(r['trace_fitness'])
    missing_tokens.append(r['missing_tokens'])
    consumed_tokens.append(r['consumed_tokens'])
    remaining_tokens.append(r['remaining_tokens'])
    produced_tokens.append(r['produced_tokens'])

# Metrics
tracefitness = np.mean(trace_fitnesses)
eventfitness = 1 - (sum(missing_tokens) / sum(consumed_tokens)) if sum(consumed_tokens) > 0 else 0
logfitness = 0.5 * (1 - (sum(missing_tokens) / sum(consumed_tokens))) + \
             0.5 * (1 - (sum(remaining_tokens) / sum(produced_tokens))) if sum(consumed_tokens) > 0 and sum(produced_tokens) > 0 else 0

print(f"Tracefitness: {tracefitness:.4f}")
print(f"Eventfitness: {eventfitness:.4f}")
print(f"Logfitness: {logfitness:.4f}")

replaying log with TBR, completed traces ::   0%|          | 0/19 [00:00<?, ?it/s]

Tracefitness: 0.8181
Eventfitness: 0.8298
Logfitness: 0.8298
